# 04. Análisis de Complejidad Algorítmica
Visualización de Big O y comparación de rendimiento.

In [ ]:
// Visualización de complejidad con SVG

function generarGraficaComplejidad() {
    const width = 600, height = 400;
    const padding = 40;
    
    // Datos: n vs operaciones para diferentes complejidades
    const datos = [];
    for (let n = 1; n <= 50; n += 2) {
        datos.push({
            n: n,
            O_log: Math.log2(n) * 10,
            O_n: n * 5,
            O_nlogn: n * Math.log2(n),
            O_n2: (n * n) / 10
        });
    }
    
    // Escalar para SVG (invertir Y porque SVG 0 es arriba)
    const maxVal = Math.max(...datos.map(d => d.O_n2));
    const scaleY = (val) => height - padding - (val / maxVal) * (height - 2 * padding);
    const scaleX = (n) => padding + (n / 50) * (width - 2 * padding);
    
    let svg = `<svg width="${width}" height="${height}" style="border: 1px solid #333; background: #fafafa;">`;
    
    // Ejes
    svg += `<line x1="${padding}" y1="${height-padding}" x2="${width-padding}" y2="${height-padding}" stroke="black" stroke-width="2"/>`;
    svg += `<line x1="${padding}" y1="${padding}" x2="${padding}" y2="${height-padding}" stroke="black" stroke-width="2"/>`;
    
    // Etiquetas
    svg += `<text x="${width/2}" y="${height-10}" text-anchor="middle" font-family="sans-serif">n (tamaño input)</text>`;
    svg += `<text x="15" y="${height/2}" text-anchor="middle" font-family="sans-serif" transform="rotate(-90, 15, ${height/2})">Operaciones</text>`;
    
    // Función para dibujar línea
    const drawLine = (key, color, width_line = 2) => {
        let path = `M ${scaleX(datos[0].n)} ${scaleY(datos[0][key])}`;
        for (let i = 1; i < datos.length; i++) {
            path += ` L ${scaleX(datos[i].n)} ${scaleY(datos[i][key])}`;
        }
        svg += `<path d="${path}" fill="none" stroke="${color}" stroke-width="${width_line}"/>`;
    };
    
    drawLine('O_log', 'green');
    drawLine('O_n', 'blue');
    drawLine('O_nlogn', 'orange');
    drawLine('O_n2', 'red');
    
    // Leyenda
    const leyenda = [
        {color: 'green', label: 'O(log n)'},
        {color: 'blue', label: 'O(n)'},
        {color: 'orange', label: 'O(n log n)'},
        {color: 'red', label: 'O(n²)'}
    ];
    
    leyenda.forEach((item, i) => {
        const y = 30 + i * 25;
        svg += `<rect x="${width-120}" y="${y-10}" width="15" height="15" fill="${item.color}"/>`;
        svg += `<text x="${width-100}" y="${y}" font-family="sans-serif" font-size="14">${item.label}</text>`;
    });
    
    svg += '</svg>';
    return svg;
}

({"text/html": generarGraficaComplejidad()});

In [ ]:
// Benchmarking empírico (medición de tiempo en ms)

function benchmark(fn, nombre, ...args) {
    const start = performance.now();
    const result = fn(...args);
    const end = performance.now();
    return {
        algoritmo: nombre,
        tiempo_ms: (end - start).toFixed(4),
        resultado_tipo: typeof result,
        n_input: args[0]?.length || args[0]
    };
}

// Algoritmos para comparar
function busquedaLineal(arr, target) {
    for (let i = 0; i < arr.length; i++) {
        if (arr[i] === target) return i;
    }
    return -1;
}

function busquedaBinaria(arr, target) {
    let left = 0, right = arr.length - 1;
    while (left <= right) {
        const mid = Math.floor((left + right) / 2);
        if (arr[mid] === target) return mid;
        if (arr[mid] < target) left = mid + 1;
        else right = mid - 1;
    }
    return -1;
}

// Generar datos de prueba
const crearArrayOrdenado = (n) => Array.from({length: n}, (_, i) => i * 2);

const tamanos = [1000, 10000, 100000, 500000];
const resultados = [];

tamanos.forEach(n => {
    const datos = crearArrayOrdenado(n);
    const target = datos[datos.length - 2]; // Buscar el penúltimo
    
    resultados.push(benchmark(busquedaLineal, 'Búsqueda Lineal O(n)', datos, target));
    resultados.push(benchmark(busquedaBinaria, 'Búsqueda Binaria O(log n)', datos, target));
});

// Tabla ASCII simple
let tabla = "Algoritmo               | n         | Tiempo (ms)\n";
tabla += "----------------------|-----------|------------\n";
resultados.forEach(r => {
    tabla += `${r.algoritmo.padEnd(22)}| ${String(r.n_input).padStart(9)} | ${r.tiempo_ms.padStart(10)}\n`;
});

({
    "text/plain": tabla,
    "application/json": {
        conclusion: "La búsqueda binaria es exponencialmente más rápida para n grandes",
        datos_crudos: resultados
    }
});